In [8]:
import pandas as pd
from stats_can import StatsCan

# Initialize StatsCan
sc = StatsCan()

# Load data
df = sc.table_to_df("14-10-0023-01")

# Clean and rename DataFrame
df_clean = df[['REF_DATE', 'Labour force characteristics', 'North American Industry Classification System (NAICS)', 'Sex', 'Age group', 'VALUE']]
df_main = df_clean.rename(columns={
    'REF_DATE': 'Year',
    'Labour force characteristics': 'Characteristics',
    'North American Industry Classification System (NAICS)': 'Industry',
    'VALUE': 'Value'
})
df_main['Year'] = df_main['Year'].astype(str)
df_main['Year'] = df_main['Year'].str[:4]

# Remove content inside square brackets from 'Industry' column
df_main['Industry'] = df_main['Industry'].str.replace(r'\[.*?\]', '', regex=True).str.strip()

df_yearly = df_main.groupby(['Year', 'Characteristics', 'Industry', 'Sex', 'Age group'], as_index=False).mean()

# Specify the path to save the CSV file
csv_file_path = "C:\\Users\\yeduk\\OneDrive\\Desktop\\Data\\Data_695\\Labour Market Characteristics\\processed_stats_canada_data.csv"  # Replace with your desired directory path
df_yearly.to_csv(csv_file_path, index=False).csv

# Output the processed DataFrame (optional)
print("Data successfully saved to", csv_file_path)
df_yearly



C:\Users\yeduk\AppData\Local\Temp\ipykernel_17208\2000813237.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_yearly = df_main.groupby(['Year', 'Characteristics', 'Industry', 'Sex', 'Age group'], as_index=False).mean()


Data successfully saved to https://raw.githubusercontent.com/Ed-Chris/For-Power-bi/main/processed_stats_canada_data.csv


,Year,Characteristics,Industry,Sex,Age group,Value
0,1976,Employment,Accommodation and food services,Both sexes,15 to 24 years,30.600000
1,1976,Employment,Accommodation and food services,Both sexes,15 years and over,75.154545
2,1976,Employment,Accommodation and food services,Both sexes,25 to 54 years,37.118182
3,1976,Employment,Accommodation and food services,Both sexes,55 years and over,7.463636
4,1976,Employment,Accommodation and food services,Females,15 to 24 years,18.318182
...,...,...,...,...,...,...
100219,2023,Unemployment rate,Wholesale trade,Females,55 years and over,NaN
100220,2023,Unemployment rate,Wholesale trade,Males,15 to 24 years,NaN
100221,2023,Unemployment rate,Wholesale trade,Males,15 years and over,2.880000
100222,2023,Unemployment rate,Wholesale trade,Males,25 to 54 years,3.350000


In [9]:
# Filter the data to include only 'Employment', 'Full time', and 'Part time' characteristics
selected_characteristics = ['Employment', 'Full-time employment', 'Part-time employment']
employment_data = df_yearly[df_yearly['Characteristics'].isin(selected_characteristics)]

# Pivot the table to have 'Sex' as columns
pivot_table = employment_data.pivot_table(index=['Year', 'Industry', 'Age group', 'Characteristics'], columns='Sex', values='Value')

# Calculate participation rate for males and females in percentage
pivot_table['Male Participation Rate (%)'] = (pivot_table['Males'] / pivot_table['Both sexes']) * 100
pivot_table['Female Participation Rate (%)'] = (pivot_table['Females'] / pivot_table['Both sexes']) * 100

# Reset the index to make it easier to export
pivot_table.reset_index(inplace=True)

# Select relevant columns
processed_data = pivot_table[['Year', 'Industry', 'Age group', 'Characteristics', 'Male Participation Rate (%)', 'Female Participation Rate (%)']]

# Calculate the difference in participation rates between male and female
processed_data['Difference (%)'] = processed_data['Male Participation Rate (%)'] - processed_data['Female Participation Rate (%)']

# Save the processed data to a new CSV file
processed_file_path = 'C:\\Users\\yeduk\\OneDrive\\Desktop\\Data\\Data_695\\Labour Market Characteristics\\processed_participation_rates.csv'
processed_data.to_csv(processed_file_path, index=False)

# Display the processed DataFrame
processed_data


C:\Users\yeduk\AppData\Local\Temp\ipykernel_17208\4280470266.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data['Difference (%)'] = processed_data['Male Participation Rate (%)'] - processed_data['Female Participation Rate (%)']


Sex,Year,Industry,Age group,Characteristics,Male Participation Rate (%),Female Participation Rate (%),Difference (%)
0,1976,Accommodation and food services,15 to 24 years,Employment,40.047534,59.863339,-19.815805
1,1976,Accommodation and food services,15 to 24 years,Full-time employment,40.384615,59.557110,-19.172494
2,1976,Accommodation and food services,15 to 24 years,Part-time employment,49.210207,60.328068,-11.117861
3,1976,Accommodation and food services,15 years and over,Employment,40.800774,59.199226,-18.398452
4,1976,Accommodation and food services,15 years and over,Full-time employment,45.223700,54.793574,-9.569874
...,...,...,...,...,...,...,...
14555,2023,Wholesale trade,25 to 54 years,Full-time employment,69.583983,30.416017,39.167965
14556,2023,Wholesale trade,25 to 54 years,Part-time employment,68.390805,82.327586,-13.936782
14557,2023,Wholesale trade,55 years and over,Employment,69.262594,37.389470,31.873124
14558,2023,Wholesale trade,55 years and over,Full-time employment,71.151747,39.366130,31.785617
